In [1]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as path_effects
from matplotlib import rcParams
from highlight_text import ax_text, fig_text
import pandas as pd

from PIL import Image
import urllib
import os

In [2]:
df = pd.read_csv("data/brier_score.csv",encoding="utf-8")
df["date"] = pd.to_datetime(df["date"])

In [ ]:
df["home_win"] = [1 if x > y else 0 for x,
                  y in zip(df["score_home"], df["score_away"])]
df["tie"] = [1 if x == y else 0 for x, y in zip(
    df["score_home"], df["score_away"])]
df["away_win"] = [1 if x < y else 0 for x,
                  y in zip(df["score_home"], df["score_away"])]
df = df.assign(
    brier_score=lambda x: 1/3 *
    ((x.prob_home-x.home_win)**2 +
     (x.prob_tie-x.tie)**2 +
     (x.prob_away-x.away_win)**2)
)
